<a href="https://colab.research.google.com/github/Battula888/FMML-LAB/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits generally gives more consistent and reliable results. This approach, often referred to as **cross-validation**, helps to mitigate the variability that can arise from the specific choice of a single validation set. Here's why:

1. **Reduction in Variance**: A single split of the data may not be representative of the overall distribution, and it could result in higher variance in the performance metrics. Averaging across multiple splits smooths out any idiosyncratic trends or biases present in a single validation set.

2. **Better Generalization**: By using multiple different subsets of the data for validation, you ensure that your model's performance is evaluated on various portions of the data, improving the estimate of how well the model generalizes to unseen data.

3. **Minimizing Overfitting to the Validation Set**: In a single split, there’s a risk of the model overfitting to the validation set, particularly if the model is complex. By averaging over multiple splits, you reduce the likelihood that your model is overly tailored to a specific subset of the data.

### How it works:
In k-fold cross-validation, for instance, the data is divided into k parts (or folds). The model is trained on k-1 folds and tested on the remaining fold. This process is repeated k times, with each fold serving as the validation set once. The final result is the average performance metric (such as accuracy) across all the splits.

### Example:
- **Without averaging**: If you evaluate on a single validation set, the accuracy might be 85%, but this could be a result of lucky or unlucky splits.
- **With averaging**: By running k-fold cross-validation (say 5-fold), you might get validation accuracies of 80%, 83%, 87%, 85%, and 90%. Averaging these gives a more reliable and consistent estimate of model performance (85% in this case), reducing the influence of a single outlier.

### Conclusion:
Averaging validation accuracy across multiple splits helps provide a more stable and consistent evaluation of your model's performance, especially when working with limited data.

2. Does it give more accurate estimate of test accuracy?
Averaging the validation accuracy across multiple splits does not directly give a **more accurate estimate of test accuracy**, but it does provide a **better estimate of how well the model is likely to perform on unseen data**. Here's a breakdown:

### Key Differences Between Validation and Test Accuracy

- **Validation Accuracy**: This is the accuracy of your model on a separate validation set (a subset of the training data) during training. Cross-validation helps improve the reliability of this estimate by averaging across multiple splits of the data.
- **Test Accuracy**: This is the performance of your model on a completely separate test set that the model has never seen during training or validation. The test set is meant to simulate the performance of your model on truly unseen data.

### Does Averaging Validation Accuracy Improve the Estimate of Test Accuracy?

- **Indirect Improvement**: By using multiple validation splits (such as in cross-validation), you get a better estimate of your model's **generalization ability** because it reduces the risk of overfitting to a specific validation set. A more reliable validation accuracy can help you choose a model or hyperparameters that are more likely to perform well on the test set.
  
  However, this does **not** directly provide a more accurate estimate of the test set performance itself. The test set is a completely separate entity that ideally should not be used during training or validation.

- **Overfitting Mitigation**: Cross-validation helps reduce the risk of overfitting to the validation set, which in turn may make the model more likely to generalize well to the test set. This makes the **validation accuracy** a more reliable indicator of how the model might perform on the test set. However, even with cross-validation, the true test performance can still differ slightly, especially if the test set is small or not perfectly representative.

### When You Get More Accurate Test Accuracy Estimates:
- **External Testing**: The most accurate estimate of test accuracy comes when you have a separate, untouched test set. If you're using cross-validation to tune models or select features, this can give you confidence that your model is likely to perform well on the test set, but you won't know the true performance on the test set until you evaluate it.

- **Large Datasets**: In scenarios with large amounts of data, cross-validation can give a better estimate of the test accuracy because each fold in the cross-validation uses a different subset of data. In these cases, the validation accuracy can serve as a reasonable proxy for test accuracy.

### Summary:

- **Cross-validation improves the consistency and reliability of your validation accuracy**, which indirectly helps in making better decisions regarding model selection and tuning.
- **Averaging validation accuracy does not directly give a more accurate estimate of test accuracy**, but it does provide a more stable estimate of model performance across different subsets of data. The actual test accuracy will only be known after evaluating the model on a completely separate test set.

3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
Yes, **increasing the number of iterations** (or folds in cross-validation) can generally lead to **a more reliable estimate** of model performance, but the effect depends on the context and specific goals. Here's a breakdown of how the number of iterations (or folds) affects the estimate:

### 1. **Effect on Stability and Variability of the Estimate**
- **Fewer iterations** (e.g., in 3-fold cross-validation) may result in a less stable estimate of performance. This is because each fold may be influenced by different data characteristics, and with fewer folds, the results are more likely to vary depending on the specific data split. This can lead to more variability in the performance estimates.
  
- **More iterations** (e.g., in 10-fold cross-validation) provide a better and more stable estimate of model performance. With more splits, the model is validated on a greater variety of data subsets, reducing the chance that a single, unrepresentative split skews the performance estimate.

### 2. **Effect on the Bias-Variance Tradeoff**
- **Higher number of iterations (folds)** typically results in a **lower bias** in the performance estimate. With more folds, the model is evaluated on more different data subsets, giving a more comprehensive view of how well it generalizes across the entire dataset.
  
- However, **increasing the number of iterations** (e.g., using a very high number of folds like 20 or more) may also lead to higher computational cost and may result in diminishing returns. Once you have enough folds (e.g., 10-fold), additional folds may not significantly improve the estimate.

### 3. **Effect on Overfitting and Underfitting**
- **More iterations** reduce the chance of **overfitting** to a particular validation set, as each fold trains and tests on different portions of the data. This can lead to a more accurate reflection of the model's generalization capability.
  
- Conversely, fewer iterations can sometimes lead to **underfitting** since the model might not have been sufficiently tested on a diverse set of data splits.

### 4. **Impact on Computational Time**
- **More iterations** take longer to run. For instance, in 10-fold cross-validation, the model is trained and tested 10 times, while in 3-fold, it is done 3 times. The trade-off between the number of folds and the computational resources needed should be considered based on the problem at hand and the size of the dataset.

### 5. **Practical Recommendations:**
- **For smaller datasets**: More folds (e.g., 10-fold or even leave-one-out cross-validation) tend to give better estimates since each fold uses a larger portion of the data for training, making each estimate less noisy.
  
- **For larger datasets**: The benefits of increasing folds diminish after a certain point. For example, going from 3-fold to 5-fold cross-validation may give a significant improvement, but going from 10-fold to 20-fold may yield only marginal gains in performance estimate quality.

### 6. **Diminishing Returns**:
- **Beyond a certain number of iterations**, the improvement in the estimate becomes minimal. For instance, after 10 folds, most of the improvement in the stability of the estimate tends to level off. Hence, there's a sweet spot, typically around 5–10 folds, for most problems. In extreme cases (such as leave-one-out cross-validation, or LOOCV), the number of folds equals the number of data points, which can be computationally expensive with limited additional benefit in performance estimation for larger datasets.

### Summary:
- **Higher number of iterations** generally leads to a **more reliable, stable estimate** of model performance by reducing variability in the evaluation results.
- For most practical purposes, **10-fold cross-validation** is a common and efficient choice, balancing computational cost and performance estimate accuracy.
- After a certain number of iterations, the **benefits level off** (diminishing returns), so there is no need to use excessively large numbers of folds unless the dataset is very small.

4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
In the context of a **very small training or validation dataset**, **increasing the number of iterations (or folds in cross-validation)** can help mitigate some challenges, but it is not a perfect solution. Here's how increasing the number of iterations interacts with small datasets, along with potential advantages and limitations:

### 1. **Benefits of Increasing Iterations with Small Datasets**

- **More Comprehensive Validation**: In a small dataset, each data point is more valuable because the model has less data to learn from. Increasing the number of iterations (or folds) allows each data point to be included in both the training and validation sets multiple times. This increases the **diversity of the validation data**, leading to a better estimate of model performance. For example, in **leave-one-out cross-validation (LOOCV)**, each point is treated as the validation set exactly once, ensuring that every piece of data contributes to model evaluation.
  
- **Reduction of Overfitting to Validation Data**: Small datasets can cause overfitting to the validation set because the model may memorize or become too finely tuned to it. By increasing the number of folds, the model is validated on multiple different subsets of the data, which can reduce this risk.

- **More Reliable Performance Estimate**: With very small datasets, a single validation set may not be representative of the full data distribution, leading to noisy or biased performance estimates. By increasing the number of folds, you improve the reliability of the performance metric by averaging over more splits of the data.

### 2. **Limitations of Increasing Iterations with Small Datasets**

- **Increased Variance**: While using more iterations gives you a more stable performance estimate, it doesn't solve the fundamental problem of limited data. With extremely small datasets, even cross-validation with many folds may still suffer from high variance in the performance estimates. For instance, in LOOCV, where the model is trained on almost all the data except one point, this can still result in a highly variable performance metric depending on which data point is left out.

- **No Substitute for More Data**: Increasing the number of folds can give a better estimate of model performance, but it doesn’t replace the need for more data. Small datasets are inherently prone to overfitting, and cross-validation can only reduce the overfitting to the validation set — it doesn't solve the underlying issue of having insufficient data to adequately train the model. More data is often the most effective way to improve model performance and reduce overfitting.

- **Computational Complexity**: In cases with small datasets, LOOCV or very high numbers of folds may not significantly help, and it could still be computationally expensive for limited gains.

### 3. **Alternative Approaches for Small Datasets**

While increasing the number of iterations in cross-validation helps to some extent, here are other strategies you can consider when working with a very small dataset:

- **Data Augmentation**: This is often used in fields like image and text classification, where you artificially expand the training data by generating new data points through transformations (such as rotating images or paraphrasing sentences). This can help create a larger training set, reducing the risk of overfitting.

- **Transfer Learning**: You can use a pre-trained model (trained on a larger dataset) and fine-tune it with your small dataset. This leverages knowledge from a larger corpus and can work well when you don't have enough data for training from scratch.

- **Regularization**: Techniques like L2 regularization (Ridge regression) or dropout (in neural networks) can help prevent overfitting when you have limited data by penalizing overly complex models.

- **Bayesian Methods**: These can incorporate prior knowledge into the model and work well with small datasets by combining the data with prior beliefs to improve predictions.

### 4. **Summary**

- **Increasing the number of folds** can help get a more stable estimate of model performance, especially in small datasets, by ensuring that each data point contributes to training and validation multiple times.
- However, it **does not replace the need for more data**. Small datasets are still prone to overfitting, and increasing iterations alone cannot fully mitigate this.
- You should consider **combining cross-validation with other techniques**, such as data augmentation, transfer learning, or regularization, to better handle small datasets.

In conclusion, while more iterations improve performance estimates and reduce overfitting to a specific validation split, the **root challenge** in small datasets is still the lack of enough data, which requires additional techniques to properly handle.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.